# Accélération du clearsky data generator
Je vais tout simplement calculer toutes les valeurs possibles à l'initialisation pour les mettre en cache

In [1]:
%cd /home/raphael/MILA/ift6759/Solar-Irradiance-Prediction
# Il faut utiliser CD pour s'assurer de rouler dans la racine i.e. '/home/raphael/MILA/ift6759/Solar-Irradiance-Prediction'
%pwd 
#%cd   #change to the dir you want 
from src.data import dataloader as dl
from src.data import metadata as meta
from src.data import config
from src.data import image
import pandas as pd
import logging
from pvlib.location import Location

bnd_coordinates  = meta.Coordinates(40.05192, -88.37309, 230)
logging.basicConfig(level=logging.ERROR)

meta_config = config.read_configuration_file("tests/data/samples/train_config_raphael.json") #Il faut ajuster les path ici!

metadata = meta.MetadataLoader(dataframe=meta_config.catalog)
#reader = image.ImageReader()
config = {}
config["LOCAL_PATH"] = "/home/raphael/MILA/ift6759/project1_data/hdf5v7_8bit/"
config["SKIP_MISSING"] = True
#dataloader = dl.DataLoader(reader, config)
metaloader= metadata.load(meta.Station.BND,bnd_coordinates, target_datetimes=meta_config.target_datetimes, night_time=False, skip_missing=True)
dataset = dl.create_dataset(metaloader, config)

/home/raphael/MILA/ift6759/Solar-Irradiance-Prediction


In [2]:
target_datetimes= pd.Series(meta_config.target_datetimes)
pd.DatetimeIndex(target_datetimes)

DatetimeIndex(['2015-01-04 19:15:00', '2012-04-20 12:15:00',
               '2011-12-08 07:00:00', '2011-11-06 04:00:00',
               '2014-08-11 08:45:00', '2010-07-07 08:00:00',
               '2014-09-20 23:45:00', '2013-11-19 20:00:00',
               '2010-12-27 16:45:00', '2012-09-10 03:45:00',
               ...
               '2012-09-09 20:15:00', '2013-06-06 08:45:00',
               '2014-06-08 20:00:00', '2015-09-22 11:30:00',
               '2011-02-18 20:45:00', '2012-01-10 22:30:00',
               '2012-06-28 20:15:00', '2014-07-10 00:15:00',
               '2010-02-24 08:00:00', '2010-12-26 03:15:00'],
              dtype='datetime64[ns]', length=189302, freq=None)

In [3]:
config

{'LOCAL_PATH': '/home/raphael/MILA/ift6759/project1_data/hdf5v7_8bit/',
 'SKIP_MISSING': True}

In [4]:
coordinates = bnd_coordinates
location = Location(
                latitude=coordinates.latitude,
                longitude=coordinates.longitude,
                altitude=coordinates.altitude,
            )

In [5]:
location

Location: 
  name: None
  latitude: 40.05192
  longitude: -88.37309
  altitude: 230
  tz: UTC

In [6]:
clearsky_values = location.get_clearsky(pd.DatetimeIndex(target_datetimes))
cache = {}

In [7]:
def _generate_cache_key(timestamp, coordinates):
    ts_str = str(timestamp)
    return f"{coordinates.latitude:.4f};{coordinates.longitude:.4f};{coordinates.altitude:.2f};{ts_str}"

In [52]:
clearsky_values["cache_key"] = clearsky_values.index.to_series().apply(_generate_cache_key, args= [coordinates])

In [53]:
def _load_values_to_cache(row):
    cache[row["cache_key"]] = row["ghi"]
    

In [54]:
clearsky_values.apply(_load_values_to_cache,axis=1)

2015-01-04 19:15:00    None
2012-04-20 12:15:00    None
2011-12-08 07:00:00    None
2011-11-06 04:00:00    None
2014-08-11 08:45:00    None
                       ... 
2012-01-10 22:30:00    None
2012-06-28 20:15:00    None
2014-07-10 00:15:00    None
2010-02-24 08:00:00    None
2010-12-26 03:15:00    None
Length: 189302, dtype: object

In [55]:
clearsky_values

,ghi,dni,dhi,cache_key
2015-01-04 19:15:00,414.271049,888.371118,41.593834,40.0519;-88.3731;230.00;2015-01-04 19:15:00
2012-04-20 12:15:00,125.895108,401.583391,42.992043,40.0519;-88.3731;230.00;2012-04-20 12:15:00
2011-12-08 07:00:00,0.000000,0.000000,0.000000,40.0519;-88.3731;230.00;2011-12-08 07:00:00
2011-11-06 04:00:00,0.000000,0.000000,0.000000,40.0519;-88.3731;230.00;2011-11-06 04:00:00
2014-08-11 08:45:00,0.000000,0.000000,0.000000,40.0519;-88.3731;230.00;2014-08-11 08:45:00
...,...,...,...,...
2012-01-10 22:30:00,8.175275,135.482600,2.906130,40.0519;-88.3731;230.00;2012-01-10 22:30:00
2012-06-28 20:15:00,790.371997,783.758460,137.179204,40.0519;-88.3731;230.00;2012-06-28 20:15:00
2014-07-10 00:15:00,98.633557,277.914910,44.418208,40.0519;-88.3731;230.00;2014-07-10 00:15:00
2010-02-24 08:00:00,0.000000,0.000000,0.000000,40.0519;-88.3731;230.00;2010-02-24 08:00:00


In [56]:
cache["40.0519;-88.3731;230.00;2015-01-04 19:15:00"]

414.27104917594124